<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M3_HyggeBERT_translation_en_da.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Intstalls
!pip install simpletransformers sacremoses datasets -q

In [ ]:
# Imports

import pandas as pd
from tqdm import tqdm
import logging

# datasets library
from datasets import load_dataset

# transformer models for translation
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

In [ ]:
# logging settings
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
# paths for translation file and data and chunk-size for translation
path_translate = 'translated.txt'
path_data = 'data.json.gz'
chunksize = 1000

In [ ]:
# translation model args
model_args = Seq2SeqArgs()
model_args.num_beams = 10
model_args.max_length = 50
model_args.use_multiprocessed_decoding = True

In [ ]:
# Initialize a Seq2SeqModel for English to Danish translation
model = Seq2SeqModel(
    encoder_decoder_type="marian",
    encoder_decoder_name="Helsinki-NLP/opus-mt-en-da",
    tokenizer = "Helsinki-NLP/opus-mt-en-da",
    args=model_args,
)

In [ ]:
# download and prepare dataset via datasets library
emotions = load_dataset("go_emotions", "raw")

In [ ]:
# transform to DF
data = emotions['train'].to_pandas()

In [ ]:
# take sample for quick prototyping
data = data.sample(n=1000)

Working with Colab I decided to run translation in chunks (no need to restart a 4h job due to disconnect etc.) and append translated lines to a file on disk/google-drive.

In [ ]:
# handy split function for chunking
def split(list_a, chunk_size):

  for i in range(0, len(list_a), chunk_size):
    yield list_a[i:i + chunk_size]

In [ ]:
# create splits (considering the last one won't be N)
text_gen = split(data.iloc[:,0].to_list(), chunksize)

In [ ]:
# progress bar
pbar = tqdm(total=len(data.iloc[:,0].to_list()))

In [ ]:
# loop creates chunks of N sentences, translates and appends to a textfile
while text_gen:
  try:
    chunk = next(text_gen)
    f = open(path_translate,'a')
    translated_chunk = model.predict(chunk)
    f.write('\n'.join(translated_chunk) + '\n')
    f.close()
    pbar.update(len(chunk))
  except StopIteration:
    pbar.close()
    break

In [ ]:
# open up the text file again
translated_f = open(path_translate,'r').readlines()

In [ ]:
# clean up and add as column to the initial file
dk_text = [line.strip() for line in translated_f]
data['text_dk'] = dk_text

In [ ]:
# write DF with translation to disk
data.to_json(path_data)